## MAX CLIQUE PROBLEM

In [1]:
import networkx as nx # крутая штука для операций с графами
import cplex
import time
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from threading import Timer # для остановки алгоритма по достижении лимита по времени

In [2]:
# читаем граф и получаем список ребер и количество вершин
def init_graph(file_name):
    edges = []
    
    with open(file_name, 'r') as file:
        for line in file:
            if line.startswith('c'):  # graph description
                pass
            elif line.startswith('p'):
                p, name, vertices_num, edges_num = line.split()
                pass
            elif line.startswith('e'):
                _, v1, v2 = line.split()
                edges.append((int(v1), int(v2)))
            else:
                continue
    return edges, int(vertices_num)

In [3]:
# Основной и единственный класс для расчетов, дальше будет много комментов
class MyClass:
    def __init__(self, graph, vertices_num, stop_time=30):
        Timer(stop_time, self.exitfunc).start() # Установим лимит времени
        self.check_time = False # флажок для остановки ветвления после лимита по времени
        self.nx_graph = nx.Graph(edges) # наш граф
        self.save_solution = None # save solution
        self.nodes = [i for i in range(1, vertices_num + 1)] # список всех вершин
        self.constraints = self.init_constraints(num_random_constraints=int(vertices_num)) # инициализируем ограничения
        self.model = self.init_model() # init cplex model
        self.add_constraints(self.constraints) # add constraints
        '''
        Теперь давайте возьмем эвристику (не всегда она нужна, но где-то очень даже). Я использовал тот алгоритм,
        про который говорили на паре (с последовательным выбором вершины и удаления всех не инцидентных ей).
        Чтобы получить эвристику побольше, взял два варианта - с максимальной степенью и максимальным цветом,
        плюс добавил рандома и инициализировал много раз (количество вершин / 2). Так ведь можно?
        '''
        self.current_maximum_clique = self.heuristic_by_rangdom_coloring_and_degree(num_of_iters=int(vertices_num))
        self.heuristic_clique = self.current_maximum_clique
        self.branch_num = 0 # Номер b&b
        print('START MAX CLIQUE: {}'.format(self.current_maximum_clique))
        print('START BRANCHING')                
    
    # ставим флажок для выхода после наступления лимита по времени
    def exitfunc(self):
        self.check_time = True
    
    # инициализируем ограничения
    def init_constraints(self, num_random_constraints):
        constraints = []
        # not connected - все вершины, которые не соединены в формате [[xi, xj], ... ]
        not_connected_edges = nx.complement(self.nx_graph).edges
        for i in not_connected_edges:
            constraints.append(sorted(list(i)))
        
        # color constraints
        '''
        Нам нужны еще ограничения, т.к. сейчас условные Xi + Xj <= 1 дают нам ограничения на Xi и Xj по 0.5, а мы хотим сделать
        их сильнее. Давайте раскрасим (на самом деле не важно как) граф и зададим ограничение: сумма всех вершин одного цвета
        <= 1, т.е. по итогу только одна из них будет входить в граф
        '''
        strategies = [nx.coloring.strategy_largest_first,
                      nx.coloring.strategy_independent_set,
                      nx.coloring.strategy_connected_sequential_bfs,
                      nx.coloring.strategy_connected_sequential_dfs,
                      nx.coloring.strategy_saturation_largest_first]
        
        for strategy in strategies:
            d = nx.coloring.greedy_color(self.nx_graph, strategy=strategy)
            for color in set(d.values()):
                temp = [key for key, value in d.items() if value == color]
                if len(temp) != 1 and sorted(temp) not in constraints:
                    constraints.append(sorted(temp))
        
        # random color constraints
        '''
        А давайте теперь много много раз инициализируем ранодмную раскраску графа и добавим все это в ограничения.
        Штука эмпирическая, но для некоторых графов работает отлично (это ведь не читерство?)
        '''
        for i in range(num_random_constraints):
            d = nx.coloring.greedy_color(self.nx_graph, strategy=nx.coloring.strategy_random_sequential)
            for color in set(d.values()):
                temp = [key for key, value in d.items() if value == color]
                if len(temp) != 1 and sorted(temp) not in constraints:
                    constraints.append(sorted(temp))        
        
        # set constraints - просто ограничения в нужный для cplex api формат
        result_constraints = []
        for constraint in constraints:
            constraint_name = ['x{}'.format(i) for i in constraint]
            result_constraints.append([constraint_name, [1] * len(constraint)])
        return result_constraints
        
    # init model - ничего такого, просто создаем модель и добавляем в нее переменные
    def init_model(self):
        obj = [1] * len(self.nodes)
        upper_bound = [1] * len(self.nodes)
        names = ['x{}'.format(i) for i in self.nodes]
        types = 'C' * len(self.nodes)
        
        model = cplex.Cplex()
        model.set_results_stream(None)        
        model.objective.set_sense(model.objective.sense.maximize)
        model.variables.add(obj=obj, ub=upper_bound,
                              names=names, types=types)
        
        
        return model
    
    # добавляем в модель ограничения, тоже все понятно
    def add_constraints(self, constraints):
        constraint_senses = 'L' * len(constraints)
        right_hand_side = [1] * len(constraints)
        constraint_names = ['x{}'.format(i) for i in range(len(constraints))]
        
        print('constraints', constraints[:2])
        print('senses', constraint_senses[:2])
        print('rhs', right_hand_side[:2])
        print('names', constraint_names[:2])
        
        self.model.linear_constraints.add(lin_expr=constraints,
                                          senses=constraint_senses,
                                          rhs=right_hand_side,
                                          names=constraint_names)
    
    # Эвристика через рандомную раскраску графа и выбор максимального цвета (рандомно), говорили про нее на лекции
    def heuristic_by_rangdom_coloring_and_degree(self, num_of_iters):   
        max_len = 0
        # random coloring
        for i in range(num_of_iters):
            K = []
            colors = dict(nx.coloring.greedy_color(self.nx_graph, strategy=nx.coloring.strategy_random_sequential))
            while len(colors) != 0 :
                max_color = max(colors.items(), key=lambda x: x[1])[1]
                random_node = random.choice([i[0] for i in colors.items() if i[1] == max_color])
                neigh = list(self.nx_graph.neighbors(random_node))
                K.append(random_node)
                colors.pop(random_node, None)
                colors = {i[0]:i[1] for i in colors.items() if i[0] in neigh}
            if len(K) > max_len:
                max_len = len(K)
                self.save_solution = K
        # random degree   
        for i in range(num_of_iters):
            K = []
            nodes = dict(self.nx_graph.degree)
            while len(nodes) != 0:
                max_degree = max(nodes.items(), key=lambda x: x[1])[1]
                random_node = random.choice([i[0] for i in nodes.items() if i[1] == max_degree])
                neigh = list(self.nx_graph.neighbors(random_node))
                K.append(random_node)
                nodes.pop(random_node, None)
                nodes = {i[0]:i[1] for i in nodes.items() if i[0] in neigh}
                if len(K) > max_len:
                    max_len = len(K)
                    self.save_solution = K
        return max_len
    
    '''
    Здесь выбираем ту переменную (Xi) по которой будем ветвиться. Я брал максимальную отличную от 1 исходя из логики:
    1) Эксперименты показали, что так лучше;
    2) Если переменная ближе к 1, то скорее всего она входит в клику и если сделаем ее равной 1, то обнулим много тех вершин,
    которые с ней не пересекаются. Таким образом, мы как будто накладываем ограничение сразу на несколько вершин.
    '''
    def get_bb_variable(self, solution):
        bbvar = [(j, i) for j, i in enumerate(solution) if i != 0.0 and not i.is_integer()]
        bbvar = max(bbvar, key = lambda x: x[1], default=(None, None))[0]
        return bbvar
    
    # добавляем ограничение когда ветвимся
    def add_constraint_bb(self, bbvar, constraint_value, current_branch):
        bb_constraint = [[[bbvar], [1.0]]]
        bb_senses = ['E']
        bb_rhs = [constraint_value]
        bb_names = ['branch_{0}'.format(current_branch)]
        self.model.linear_constraints.add(lin_expr=bb_constraint, senses=bb_senses,
                                          rhs=bb_rhs, names=bb_names)
    # получаем решение системы
    def get_solve(self):
        self.model.solve()
        return self.model.solution.get_values()
    
    # здесь все ветвление
    def branch_and_bounds(self):
        if self.check_time != True:
            # get current solution
            solution = self.get_solve()
            # working with epsilon - я взял 1e-9, наверное этого достаточно
            solution = [0.0  if i - 1e-9 <= 0.0 else 1.0 if i + 1e-9 >= 1.0 else i for i in solution]

            # Если текущее решение больше тлт равно уже найденной клики + 1.
            '''
            Я беру текущую клику + 1 т.к. нам нужно целочисленное решение и например если у нас есть клика размера 12,
            то все решения < 13 нам не нужны, т.к. максимум они дадут 12.
            - 1e-9 стоит из-за каких-то технических особенностей, которые выявились на графе gen200_p0.9_44.clq
            '''
            if sum(solution) >= self.current_maximum_clique + 1 - 1e-9:
                bbvar = self.get_bb_variable(solution)
                # Если мы нашли integer decision
                if bbvar is None:
                    self.current_maximum_clique = sum(solution)
                    self.save_solution = solution
                    print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
                    return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.constraints
                # Если решение не целочисленное
                else:
                    self.branch_num += 1 # увеличиваем ветвление на 1
                    current_branch = self.branch_num
                    '''
                    Здесь начинаем ветвиться. Я вначале беру ветку где Xi == 1, чисто эмпирическая штука, полученная
                    в результате экспериментов
                    '''
                    self.add_constraint_bb(bbvar, 1.0, current_branch) # добавляем ограничение
                    branch_1 = self.branch_and_bounds() # создаем первую ветку
                    self.model.linear_constraints.delete('branch_{0}'.format(current_branch)) # удаляем ее ограничения
                    self.add_constraint_bb(bbvar, 0.0, current_branch) # добавляем ограничение
                    branch_2 = self.branch_and_bounds() # создаем вторую ветку
                    self.model.linear_constraints.delete('branch_{0}'.format(current_branch)) # удаляем ее ограничения
                    return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.constraints       
            else:
                return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.constraints

In [4]:
edges, vertices_num = init_graph('dimacs graphs/keller4.clq')
res = MyClass(edges, vertices_num, stop_time=60*60).branch_and_bounds()

constraints [[['x6', 'x7'], [1, 1]], [['x5', 'x6'], [1, 1]]]
senses LL
rhs [1, 1]
names ['x0', 'x1']
START MAX CLIQUE: 11
START BRANCHING


In [5]:
edges, vertices_num = init_graph('dimacs graphs/' + 'brock200_2.clq')

### СИСТЕМА

##### Intel i5-8300H CPU @2.30GHz
#### RAM 16.0 GB

#### По обозначениям колонок:
- Dimacs Name - имя файла с графом
- Max Clique - реальная максимальная клика
- Time BB - сколько алгоритм работал в секундах
- FInded CLique - найденная клика
- Branches Count - сколько было ветвлений
- Heuristic - начальная эвристика
- Time BB Minutes - Сколько работал в минутах

### Easy Graphs

In [ ]:
# EASY GRAPHS
result = []
solutions = []
models, constraints = [], []

dimacs_graphs = [['p_hat300-1.clq', 8], ['C125.9.clq', 34],
                 ['brock200_2.clq', 12], ['keller4.clq', 11]]
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}, FINDING CLIQUE: {}'.format(dimacs_graph[0], dimacs_graph[1]))
    temp = [dimacs_graph[0], dimacs_graph[1]]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    res = MyClass(edges, vertices_num, stop_time=60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
        models.append(res[4])
        constraints.append(res[5])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: p_hat300-1.clq, FINDING CLIQUE: 8
START MAX CLIQUE: 8
START BRANCHING
END TIME: 379.96370697021484

DIMACS GRAPH: C125.9.clq, FINDING CLIQUE: 34
START MAX CLIQUE: 31
START BRANCHING
MAX CLIQUE: 34.0


In [5]:
# давайте проверим те графы, которые нашел алгоритм
finded_cliques = {'p_hat300-1.clq': solutions[0], # где-то эвристикой находит и там сразу вершины
                  'C125.9.clq': np.where(np.array(solutions[1]) == 1.0)[0] + 1, # где-то через solution выводить приходится
                  'brock200_2.clq': np.where(np.array(solutions[2]) == 1.0)[0] + 1,
                  'keller4.clq': solutions[3]}
dimacs_graphs = [['p_hat300-1.clq', 8], ['C125.9.clq', 34],
                 ['brock200_2.clq', 12], ['keller4.clq', 11]]

for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph[0]))
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    sol = finded_cliques[dimacs_graph[0]]
    print(sol)
    for i in sol:
        for j in sol:
            if i == j:
                continue
            assert (i, j) in edges or (j, i) in edges
    print()

DIMACS GRAPH: p_hat300-1.clq
[79, 208, 38, 159, 240, 268, 219, 250]

DIMACS GRAPH: C125.9.clq
[  1   2   5   7   9  11  17  18  19  25  29  31  34  40  44  45  48  49
  54  70  71  77  79  80  98  99 101 110 114 115 117 121 122 125]

DIMACS GRAPH: brock200_2.clq
[ 27  48  55  70 105 120 121 135 145 149 158 183]

DIMACS GRAPH: keller4.clq
[4, 45, 153, 50, 56, 138, 27, 78, 102, 162, 72]



In [6]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,p_hat300-1.clq,8,386,8.0,1634,8,6.43
1,C125.9.clq,34,211,34.0,13345,32,3.52
2,brock200_2.clq,12,185,12.0,1886,9,3.08
3,keller4.clq,11,102,11.0,1574,11,1.70


Круто, мы можем найти все клики, причем максимум мы около 6.5 минут тратим на то чтобы выйти из алгоритма. Где-то даже эвристикой можем это сделать

### Старые эксперименты, пересчитывать долго да и незачем, все равно скорость алгоритма не выросла

In [25]:
# Здесь попробуем подняться на уровень выше в прошлых графах, посмотрим посчитаются ли она за час
result = []

dimacs_graphs = [['C250.9.clq', 44], ['brock200_4.clq', 17],
                 ['keller5.clq', 27], ['p_hat300-2.clq', 25]]
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}, FINDING CLIQUE: {}'.format(dimacs_graph[0], dimacs_graph[1]))
    temp = [dimacs_graph[0], dimacs_graph[1]]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    res = MyClass(edges, vertices_num, stop_time=60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    temp.append(res[0])
    temp.append(res[1])
    temp.append(res[2])
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: C250.9.clq, FINDING CLIQUE: 44
START MAX CLIQUE: 37
START BRANCHING
MAX CLIQUE: 38.0
MAX CLIQUE: 39.0
MAX CLIQUE: 40.0
MAX CLIQUE: 41.0
MAX CLIQUE: 42.0
MAX CLIQUE: 43.0
END TIME: 3600.0680482387543

DIMACS GRAPH: brock200_4.clq, FINDING CLIQUE: 17
START MAX CLIQUE: 14
START BRANCHING
MAX CLIQUE: 15.0
MAX CLIQUE: 16.0
MAX CLIQUE: 17.0
END TIME: 886.1531031131744

DIMACS GRAPH: keller5.clq, FINDING CLIQUE: 27
START MAX CLIQUE: 21
START BRANCHING
MAX CLIQUE: 27.0
END TIME: 3600.51619887352

DIMACS GRAPH: p_hat300-2.clq, FINDING CLIQUE: 25
START MAX CLIQUE: 24
START BRANCHING
MAX CLIQUE: 25.0
END TIME: 1648.5363578796387



In [26]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,C250.9.clq,44,3600,43.0,81954,37,60.00
1,brock200_4.clq,17,886,17.0,8850,14,14.77
2,keller5.clq,27,3600,27.0,2173,21,60.00
3,p_hat300-2.clq,25,1648,25.0,3491,24,27.47


Здесь уже результаты похуже, только два графа отработали хорошо. В первом графе не нашел за час максимальную клику, а в третьем максимальную клику нашел, но за час не закончил ветвиться

In [8]:
# Попробуем  MANN
result = []

dimacs_graphs = [['MANN_a27.clq', 126], ['MANN_a45.clq', 345]]
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}, FINDING CLIQUE: {}'.format(dimacs_graph[0], dimacs_graph[1]))
    temp = [dimacs_graph[0], dimacs_graph[1]]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    res = MyClass(edges, vertices_num, stop_time=4*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: MANN_a27.clq, FINDING CLIQUE: 126
START MAX CLIQUE: 125
START BRANCHING
MAX CLIQUE: 126.0
END TIME: 93.30464386940002

DIMACS GRAPH: MANN_a45.clq, FINDING CLIQUE: 345
START MAX CLIQUE: 341
START BRANCHING
MAX CLIQUE: 342.0
MAX CLIQUE: 343.0
MAX CLIQUE: 344.0
MAX CLIQUE: 345.0
END TIME: 3006.246346950531



In [9]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,MANN_a27.clq,126,93,126.0,2676,125,1.55
1,MANN_a45.clq,345,3006,345.0,62404,341,50.10


Вроде неплохо считается

### Gen200 - проблемный граф, который давал ошибку

In [4]:
# Попробуем  gen200
result = []
solutions = []
models, constraints = [], []

dimacs_graphs = [['gen200_p0.9_44.clq', 44], ['gen200_p0.9_55.clq', 55]]
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}, FINDING CLIQUE: {}'.format(dimacs_graph[0], dimacs_graph[1]))
    temp = [dimacs_graph[0], dimacs_graph[1]]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    res = MyClass(edges, vertices_num, stop_time=60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
        models.append(res[4])
        constraints.append(res[5])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: gen200_p0.9_44.clq, FINDING CLIQUE: 44
START MAX CLIQUE: 35
START BRANCHING
MAX CLIQUE: 38.0
MAX CLIQUE: 39.0
MAX CLIQUE: 40.0
MAX CLIQUE: 41.0
MAX CLIQUE: 42.0
MAX CLIQUE: 43.0
MAX CLIQUE: 44.0
END TIME: 1260.7479314804077

DIMACS GRAPH: gen200_p0.9_55.clq, FINDING CLIQUE: 55
START MAX CLIQUE: 40
START BRANCHING
MAX CLIQUE: 55.0
END TIME: 3.811457395553589



In [5]:
# давайте проверим те графы, которые нашел алгоритм
finded_cliques = {'gen200_p0.9_44.clq': np.where(np.array(solutions[0]) == 1.0)[0] + 1, 
                  'gen200_p0.9_55.clq': np.where(np.array(solutions[1]) == 1.0)[0] + 1}
dimacs_graphs = [['gen200_p0.9_44.clq', 44], ['gen200_p0.9_55.clq', 55]]

for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph[0]))
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    sol = finded_cliques[dimacs_graph[0]]
    print(sol)
    for i in sol:
        for j in sol:
            if i == j:
                continue
            assert (i, j) in edges or (j, i) in edges
    print()

DIMACS GRAPH: gen200_p0.9_44.clq
[  2  13  20  29  34  38  40  46  47  65  67  72  75  81  82  84  93  94
  97 100 102 105 108 117 119 120 123 127 129 132 138 141 146 149 150 151
 156 166 170 180 186 190 193 195]

DIMACS GRAPH: gen200_p0.9_55.clq
[  5   6  12  14  15  19  21  25  26  27  30  33  35  36  41  62  64  67
  69  73  76  77  78  79  81  82  86  88  89  91  93  95  96 107 111 113
 116 117 123 129 143 144 146 147 159 163 164 169 175 177 182 187 192 197
 199]



In [6]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,gen200_p0.9_44.clq,44,1260,44.0,34142,35,21.00
1,gen200_p0.9_55.clq,55,3,55.0,0,40,0.05


Второй граф за счет ограничений даже не ветвится, а сразу находит целочисленное решение

#### До этого gen200_p0.9_44.clq выдавал ошибку (как решил написал в письме), давайте лучше пять раза проверим, что там все ок, а то вдруг

In [5]:
# Попробуем  gen200, который выдавал ошибку 3 РАЗА
dimacs_graphs = [['gen200_p0.9_44.clq', 44], ['gen200_p0.9_44.clq', 44], ['gen200_p0.9_44.clq', 44],
                 ['gen200_p0.9_44.clq', 44], ['gen200_p0.9_44.clq', 44]]
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}, FINDING CLIQUE: {}'.format(dimacs_graph[0], dimacs_graph[1]))
    temp = [dimacs_graph[0], dimacs_graph[1]]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    res = MyClass(edges, vertices_num, stop_time=4*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: gen200_p0.9_44.clq, FINDING CLIQUE: 44
START MAX CLIQUE: 36
START BRANCHING
MAX CLIQUE: 38.0
MAX CLIQUE: 39.0
MAX CLIQUE: 40.0
MAX CLIQUE: 41.0
MAX CLIQUE: 42.0
MAX CLIQUE: 43.0
MAX CLIQUE: 44.0
END TIME: 3133.2420547008514

DIMACS GRAPH: gen200_p0.9_44.clq, FINDING CLIQUE: 44
START MAX CLIQUE: 35
START BRANCHING
MAX CLIQUE: 36.0
MAX CLIQUE: 37.0
MAX CLIQUE: 39.0
MAX CLIQUE: 40.0
MAX CLIQUE: 41.0
MAX CLIQUE: 42.0
MAX CLIQUE: 43.0
MAX CLIQUE: 44.0
END TIME: 142.4429898262024

DIMACS GRAPH: gen200_p0.9_44.clq, FINDING CLIQUE: 44
START MAX CLIQUE: 37
START BRANCHING
MAX CLIQUE: 38.0
MAX CLIQUE: 39.0
MAX CLIQUE: 40.0
MAX CLIQUE: 41.0
MAX CLIQUE: 42.0
MAX CLIQUE: 43.0
MAX CLIQUE: 44.0
END TIME: 1592.0018372535706

DIMACS GRAPH: gen200_p0.9_44.clq, FINDING CLIQUE: 44
START MAX CLIQUE: 36
START BRANCHING
MAX CLIQUE: 38.0
MAX CLIQUE: 39.0
MAX CLIQUE: 40.0
MAX CLIQUE: 41.0
MAX CLIQUE: 42.0
MAX CLIQUE: 43.0
MAX CLIQUE: 44.0
END TIME: 934.3558287620544

DIMACS GRAPH: gen200_p0.9_44.

Вроде все работает, плюс виден большой разброс в зависимости от рандома (это больше данного графа фишка, т.к. там очень мало веток где решение больше максимальной клики, для например первых четырех маленьких графов разброс намного меньше)

## Это графы из последнего письма (вашего), там часть повторяется, но ладно, пусть будет. Буду их по семействам запускать часа на два, а там посмотрим. 

### P.S. Максимальная клика (решение задачи) не везде есть, так что забил на нее

### brock200 (был до этого, так что оставил его)

In [7]:
# давайте посмотрим на brock200
result = []
solutions = []

dimacs_graphs = [['brock200_4.clq', 17], ['brock200_3.clq', 15],
                 ['brock200_2.clq', 12], ['brock200_1.clq', 21]]
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}, FINDING CLIQUE: {}'.format(dimacs_graph[0], dimacs_graph[1]))
    temp = [dimacs_graph[0], dimacs_graph[1]]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    res = MyClass(edges, vertices_num, stop_time=2*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: brock200_4.clq, FINDING CLIQUE: 17
START MAX CLIQUE: 14
START BRANCHING
MAX CLIQUE: 16.0
MAX CLIQUE: 17.0
END TIME: 872.1187179088593

DIMACS GRAPH: brock200_3.clq, FINDING CLIQUE: 15
START MAX CLIQUE: 13
START BRANCHING
MAX CLIQUE: 14.0
MAX CLIQUE: 15.0
END TIME: 584.5554914474487

DIMACS GRAPH: brock200_2.clq, FINDING CLIQUE: 12
START MAX CLIQUE: 10
START BRANCHING
MAX CLIQUE: 11.0
MAX CLIQUE: 12.0
END TIME: 270.15586829185486

DIMACS GRAPH: brock200_1.clq, FINDING CLIQUE: 21
START MAX CLIQUE: 18
START BRANCHING
MAX CLIQUE: 20.0
MAX CLIQUE: 21.0
END TIME: 3503.628757953644



In [8]:
# давайте проверим те графы, которые нашел алгоритм
finded_cliques = {'brock200_4.clq': np.where(np.array(solutions[0]) == 1.0)[0] + 1,
                  'brock200_3.clq': np.where(np.array(solutions[1]) == 1.0)[0] + 1,
                  'brock200_2.clq': np.where(np.array(solutions[2]) == 1.0)[0] + 1,
                  'brock200_1.clq': np.where(np.array(solutions[3]) == 1.0)[0] + 1}
dimacs_graphs = [['brock200_4.clq', 17], ['brock200_3.clq', 15],
                 ['brock200_2.clq', 12], ['brock200_1.clq', 21]]

for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph[0]))
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    sol = finded_cliques[dimacs_graph[0]]
    print(sol)
    for i in sol:
        for j in sol:
            if i == j:
                continue
            assert (i, j) in edges or (j, i) in edges
    print()

DIMACS GRAPH: brock200_4.clq
[ 12  19  28  29  38  54  65  71  79  93 117 127 139 161 165 186 192]

DIMACS GRAPH: brock200_3.clq
[ 12  29  36  38  58  84  97  98 104 118 130 144 158 173 178]

DIMACS GRAPH: brock200_2.clq
[ 27  48  55  70 105 120 121 135 145 149 158 183]

DIMACS GRAPH: brock200_1.clq
[  4  26  32  41  46  48  83 100 103 104 107 120 122 132 137 138 144 175
 180 191 199]



In [9]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,brock200_4.clq,17,872,17.0,9143,14,14.53
1,brock200_3.clq,15,584,15.0,6390,13,9.73
2,brock200_2.clq,12,270,12.0,3286,10,4.50
3,brock200_1.clq,21,3503,21.0,40862,18,58.38


### c-fat

In [4]:
result = []
solutions = []

dimacs_graphs = ['c-fat200-1.clq', 'c-fat200-2.clq', 'c-fat200-5.clq', 'c-fat500-1.clq', 
                 'c-fat500-10.clq', 'c-fat500-2.clq', 'c-fat500-5.clq']
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    temp = [dimacs_graph]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    res = MyClass(edges, vertices_num, stop_time=2*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: c-fat200-1.clq
START MAX CLIQUE: 12
START BRANCHING
END TIME: 1.6009821891784668

DIMACS GRAPH: c-fat200-2.clq
START MAX CLIQUE: 24
START BRANCHING
END TIME: 2.2049829959869385

DIMACS GRAPH: c-fat200-5.clq
START MAX CLIQUE: 58
START BRANCHING
END TIME: 14.88728642463684

DIMACS GRAPH: c-fat500-1.clq
START MAX CLIQUE: 14
START BRANCHING
END TIME: 17.86211657524109

DIMACS GRAPH: c-fat500-10.clq
START MAX CLIQUE: 126
START BRANCHING
END TIME: 151.66854643821716

DIMACS GRAPH: c-fat500-2.clq
START MAX CLIQUE: 26
START BRANCHING
END TIME: 31.229766130447388

DIMACS GRAPH: c-fat500-5.clq
START MAX CLIQUE: 64
START BRANCHING
END TIME: 68.18281435966492



In [5]:
result1 = result
solutions1 = solutions

In [20]:
# давайте проверим те графы, которые нашел алгоритм
finded_cliques = {'c-fat200-1.clq': solutions[0],
                  'c-fat200-2.clq': solutions[1],
                  'c-fat200-5.clq': solutions[2],
                  'c-fat500-1.clq': solutions[3],
                  'c-fat500-10.clq': solutions[4],
                  'c-fat500-2.clq': solutions[5],
                  'c-fat500-5.clq': solutions[6]}
dimacs_graphs = ['c-fat200-1.clq', 'c-fat200-2.clq', 'c-fat200-5.clq', 'c-fat500-1.clq', 
                 'c-fat500-10.clq', 'c-fat500-2.clq', 'c-fat500-5.clq']

for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    sol = finded_cliques[dimacs_graph]
    print(sol)
    for i in sol:
        for j in sol:
            if i == j:
                continue
            assert (i, j) in edges or (j, i) in edges
    print()

DIMACS GRAPH: c-fat200-1.clq
[45, 46, 82, 120, 194, 119, 157, 83, 193, 8, 156, 9]

DIMACS GRAPH: c-fat200-2.clq
[182, 91, 38, 163, 37, 128, 145, 92, 181, 2, 199, 127, 55, 73, 110, 74, 20, 164, 19, 56, 200, 109, 146, 1]

DIMACS GRAPH: c-fat200-5.clq
[157, 23, 136, 51, 177, 3, 156, 66, 86, 121, 65, 114, 129, 72, 170, 31, 115, 45, 80, 73, 58, 101, 17, 143, 10, 149, 52, 9, 178, 24, 108, 199, 164, 184, 59, 37, 38, 122, 185, 94, 93, 87, 30, 107, 191, 44, 163, 79, 100, 2, 198, 135, 192, 16, 142, 128, 150, 171]

DIMACS GRAPH: c-fat500-1.clq
[258, 418, 19, 99, 98, 498, 338, 419, 178, 179, 499, 18, 259, 339]

DIMACS GRAPH: c-fat500-10.clq
[427, 475, 44, 20, 412, 419, 420, 107, 380, 3, 467, 491, 36, 140, 283, 156, 11, 276, 284, 212, 244, 187, 60, 76, 460, 164, 155, 411, 324, 188, 43, 308, 35, 492, 179, 59, 84, 259, 4, 67, 444, 363, 219, 252, 251, 204, 12, 300, 68, 132, 315, 100, 292, 443, 484, 500, 147, 180, 291, 172, 275, 387, 75, 139, 332, 83, 307, 28, 27, 435, 468, 51, 124, 348, 52, 436, 339, 

In [44]:
result_df = pd.DataFrame(result1, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,c-fat200-1.clq,-,1,12,0,12,0.02
1,c-fat200-2.clq,-,2,24,0,24,0.03
2,c-fat200-5.clq,-,14,58,24,58,0.23
3,c-fat500-1.clq,-,17,14,0,14,0.28
4,c-fat500-10.clq,-,151,126,0,126,2.52
5,c-fat500-2.clq,-,31,26,0,26,0.52
6,c-fat500-5.clq,-,68,64,0,64,1.13


Такие вот они сильно разреженные графы, пришлось даже алгоритм немного переписать, а то они почти все без ветвления решаются (даже правильно). Бывает правда что инициализация долгая, но да ладно

### san

In [22]:
result = []
solutions = []

dimacs_graphs = ['san200_0.7_1.clq', 'san200_0.7_2.clq', 'san200_0.9_1.clq', 'san200_0.9_2.clq', 
                 'san200_0.9_3.clq', 'sanr200_0.7.clq']
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    temp = [dimacs_graph, dimacs_graph
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    res = MyClass(edges, vertices_num, stop_time=2*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: san200_0.7_1.clq
START MAX CLIQUE: 18
START BRANCHING
MAX CLIQUE: 30.0
END TIME: 2.948136806488037

DIMACS GRAPH: san200_0.7_2.clq
START MAX CLIQUE: 15
START BRANCHING
MAX CLIQUE: 18.0
END TIME: 8.802275657653809

DIMACS GRAPH: san200_0.9_1.clq
START MAX CLIQUE: 49
START BRANCHING
MAX CLIQUE: 70.0
END TIME: 4.522169351577759

DIMACS GRAPH: san200_0.9_2.clq
START MAX CLIQUE: 42
START BRANCHING
MAX CLIQUE: 60.0
END TIME: 3.7546279430389404

DIMACS GRAPH: san200_0.9_3.clq
START MAX CLIQUE: 32
START BRANCHING
MAX CLIQUE: 33.0
MAX CLIQUE: 34.0
MAX CLIQUE: 35.0
MAX CLIQUE: 38.0
MAX CLIQUE: 39.0
MAX CLIQUE: 40.0
MAX CLIQUE: 41.0
MAX CLIQUE: 42.0
MAX CLIQUE: 44.0
END TIME: 74.29857277870178

DIMACS GRAPH: sanr200_0.7.clq
START MAX CLIQUE: 15
START BRANCHING
MAX CLIQUE: 17.0
MAX CLIQUE: 18.0
END TIME: 1677.1736345291138



In [23]:
result2 = result
solutions2 = solutions

In [34]:
# давайте проверим те графы, которые нашел алгоритм
finded_cliques = {'san200_0.7_1.clq': np.where(np.array(solutions[0]) == 1.0)[0] + 1,
                  'san200_0.7_2.clq': np.where(np.array(solutions[1]) == 1.0)[0] + 1,
                  'san200_0.9_1.clq': np.where(np.array(solutions[2]) == 1.0)[0] + 1,
                  'san200_0.9_2.clq': np.where(np.array(solutions[3]) == 1.0)[0] + 1,
                  'san200_0.9_3.clq': np.where(np.array(solutions[4]) == 1.0)[0] + 1,
                  'sanr200_0.7.clq': np.where(np.array(solutions[5]) == 1.0)[0] + 1}
dimacs_graphs = ['san200_0.7_1.clq', 'san200_0.7_2.clq', 'san200_0.9_1.clq', 'san200_0.9_2.clq', 
                 'san200_0.9_3.clq', 'sanr200_0.7.clq']

for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    sol = finded_cliques[dimacs_graph]
    print(sol)
    for i in sol:
        for j in sol:
            if i == j:
                continue
            assert (i, j) in edges or (j, i) in edges
    print()

DIMACS GRAPH: san200_0.7_1.clq
[  2  12  16  19  31  47  49  57  72  81  98 101 111 123 131 136 138 141
 142 150 152 157 160 161 163 171 172 175 176 196]

DIMACS GRAPH: san200_0.7_2.clq
[  4  13  15  23  26  37  51  68  70 111 117 149 162 164 179 180 192 199]

DIMACS GRAPH: san200_0.9_1.clq
[  1   2   5   7   8  11  15  16  21  22  23  24  26  27  32  33  38  39
  40  43  48  53  54  56  60  61  62  68  69  71  77  80  92  93  94 104
 108 109 110 119 120 121 125 136 142 146 147 151 155 158 159 160 162 163
 164 165 167 170 171 172 173 176 183 186 187 189 193 195 198 200]

DIMACS GRAPH: san200_0.9_2.clq
[  4  10  12  15  17  19  23  24  26  30  33  37  38  46  51  52  56  57
  58  59  73  74  75  76  79  83  86  88  92  94  98  99 105 112 114 129
 131 133 135 136 138 141 144 149 159 160 161 163 165 167 168 169 170 174
 177 179 182 186 195 196]

DIMACS GRAPH: san200_0.9_3.clq
[  8  14  20  27  34  37  40  44  47  49  50  60  67  68  74  78  80  87
  89  94 100 101 102 105 111 113 117 119 

In [48]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,san200_0.7_1.clq,-,2,30.0,0,18,0.03
1,san200_0.7_2.clq,-,8,18.0,84,15,0.13
2,san200_0.9_1.clq,-,4,70.0,0,49,0.07
3,san200_0.9_2.clq,-,3,60.0,0,42,0.05
4,san200_0.9_3.clq,-,74,44.0,4714,32,1.23
5,sanr200_0.7.clq,-,1677,18.0,18738,15,27.95


### Ну и остальное до кучи

In [24]:
result = []
solutions = []

dimacs_graphs = ['MANN_a9.clq', 'hamming6-2.clq', 'hamming6-4.clq', 'gen200_p0.9_44.clq', 
                 'gen200_p0.9_55.clq', 'C125.9.clq', 'keller4.clq', 'p_hat300-1.clq',
                 'p_hat300-2.clq']
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    temp = [dimacs_graph]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    res = MyClass(edges, vertices_num, stop_time=2*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: MANN_a9.clq
START MAX CLIQUE: 16
START BRANCHING
END TIME: 0.13697075843811035

DIMACS GRAPH: hamming6-2.clq
START MAX CLIQUE: 32
START BRANCHING
END TIME: 0.18215513229370117

DIMACS GRAPH: hamming6-4.clq
START MAX CLIQUE: 4
START BRANCHING
END TIME: 0.4584779739379883

DIMACS GRAPH: gen200_p0.9_44.clq 
START MAX CLIQUE: 36
START BRANCHING
MAX CLIQUE: 37.0
MAX CLIQUE: 38.0
MAX CLIQUE: 39.0
MAX CLIQUE: 40.0
MAX CLIQUE: 41.0
MAX CLIQUE: 42.0
MAX CLIQUE: 43.0
MAX CLIQUE: 44.0
END TIME: 2104.865364551544

DIMACS GRAPH: gen200_p0.9_55.clq
START MAX CLIQUE: 48
START BRANCHING
MAX CLIQUE: 55.0
END TIME: 3.934717893600464

DIMACS GRAPH: C125.9.clq
START MAX CLIQUE: 31
START BRANCHING
MAX CLIQUE: 33.0
MAX CLIQUE: 34.0
END TIME: 252.59700894355774

DIMACS GRAPH: keller4.clq
START MAX CLIQUE: 11
START BRANCHING
END TIME: 95.06414031982422

DIMACS GRAPH: p_hat300-1.clq
START MAX CLIQUE: 8
START BRANCHING
END TIME: 381.47945261001587

DIMACS GRAPH: p_hat300-2.clq
START MAX CLIQUE: 24

In [25]:
result3 = result
solutions3 = solutions

In [60]:
# давайте проверим те графы, которые нашел алгоритм
finded_cliques = {'MANN_a9.clq': solutions[0],
                  'hamming6-2.clq': solutions[1],
                  'hamming6-4.clq': solutions[2],
                  'gen200_p0.9_44.clq': np.where(np.array(solutions[3]) == 1.0)[0] + 1,
                  'gen200_p0.9_55.clq': np.where(np.array(solutions[4]) == 1.0)[0] + 1,
                  'C125.9.clq': np.where(np.array(solutions[5]) == 1.0)[0] + 1,
                  'keller4.clq': solutions[6],
                  'p_hat300-1.clq': solutions[7],
                  'p_hat300-2.clq': np.where(np.array(solutions[8]) == 1.0)[0] + 1}
dimacs_graphs = ['MANN_a9.clq', 'hamming6-2.clq', 'hamming6-4.clq', 'gen200_p0.9_44.clq', 
                 'gen200_p0.9_55.clq', 'C125.9.clq', 'keller4.clq', 'p_hat300-1.clq',
                 'p_hat300-2.clq']

for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    sol = finded_cliques[dimacs_graph]
    print(sol)
    for i in sol:
        for j in sol:
            if i == j:
                continue
            assert (i, j) in edges or (j, i) in edges
    print()

DIMACS GRAPH: MANN_a9.clq
[5, 2, 1, 6, 42, 45, 24, 17, 13, 33, 12, 9, 29, 37, 36, 27]

DIMACS GRAPH: hamming6-2.clq
[23, 57, 50, 43, 42, 26, 38, 8, 2, 12, 3, 60, 22, 51, 5, 45, 29, 39, 20, 15, 48, 36, 17, 27, 53, 63, 33, 9, 56, 32, 62, 14]

DIMACS GRAPH: hamming6-4.clq
[40, 10, 21, 59]

DIMACS GRAPH: gen200_p0.9_44.clq
[ 13  20  29  34  38  40  42  46  47  65  67  72  75  81  82  84  93  94
  97 100 102 105 108 117 119 120 123 127 129 132 138 141 146 149 150 151
 156 166 170 180 186 190 193 195]

DIMACS GRAPH: gen200_p0.9_55.clq
[  5   6  12  14  15  19  21  25  26  27  30  33  35  36  41  62  64  67
  69  73  76  77  78  79  81  82  86  88  89  91  93  95  96 107 111 113
 116 117 123 129 143 144 146 147 159 163 164 175 177 182 187 192 197 198
 199]

DIMACS GRAPH: C125.9.clq
[  1   5   7   9  11  13  17  19  25  29  31  34  40  44  45  49  52  54
  55  66  70  77  79  80  96  98  99 103 104 110 114 117 122 125]

DIMACS GRAPH: keller4.clq
[13, 56, 40, 100, 68, 167, 74, 120, 152, 147, 9]

In [63]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,MANN_a9.clq,-,0,16.0,6,16,0.00
1,hamming6-2.clq,-,0,32.0,0,32,0.00
2,hamming6-4.clq,-,0,4.0,19,4,0.00
3,gen200_p0.9_44.clq,-,2104,44.0,56539,36,35.07
4,gen200_p0.9_55.clq,-,3,55.0,0,48,0.05
5,C125.9.clq,-,252,34.0,13876,31,4.20
6,keller4.clq,-,95,11.0,1501,11,1.58
7,p_hat300-1.clq,-,381,8.0,1679,8,6.35
8,p_hat300-2.clq,-,1637,25.0,3485,24,27.28


## Не везде в файлах написаны максимальные клики, давайте посчитаем сами через алгоритм на целых числах

In [4]:
from docplex.mp.model import Model

In [5]:
def fit_model(edges, vertices_num):
    N = [i for i in range(1, int(vertices_num)+1)]
    # init model
    mdl = Model('my_model')
    # init vinaries dict
    x = mdl.binary_var_dict(N, name='x')
    c = mdl.binary_var_dict(edges, name='c')
    # objective function
    mdl.maximize(mdl.sum(x[i] for i in N))
    
    # constraints
    start_time = time.time()
    constraints = mdl.add_constraints(x[i] <= 1 - x[j] for i in N for j in N if (i,j) not in edges and (j,i) not in edges and i != j)
    # get solution
    start_time = time.time()
    mdl.solve()
    return mdl

In [ ]:
dimacs_graphs = ['brock200_4.clq', 'brock200_3.clq', 'brock200_2.clq', 'brock200_1.clq',
                 'c-fat200-1.clq', 'c-fat200-2.clq', 'c-fat200-5.clq', 'c-fat500-1.clq',  'c-fat500-10.clq',
                 'c-fat500-2.clq', 'c-fat500-5.clq', 'san200_0.7_1.clq', 'san200_0.7_2.clq', 'san200_0.9_1.clq',
                 'san200_0.9_2.clq', 'san200_0.9_3.clq', 'sanr200_0.7.clq', 'MANN_a9.clq', 'hamming6-2.clq',
                 'hamming6-4.clq', 'gen200_p0.9_44.clq', 'gen200_p0.9_55.clq', 'C125.9.clq', 'keller4.clq',
                 'p_hat300-1.clq', 'p_hat300-2.clq']
real_max_cliques = []

for dimacs_graph in tqdm(dimacs_graphs):
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    res = fit_model(edges, vertices_num)
    real_max_cliques.append([dimacs_graph, res.objective_value])

  4%|▍         | 1/26 [01:06<27:51, 66.85s/it]

In [ ]:
dimacs_graphs = ['p_hat300-3.clq']
real_max_cliques = []

for dimacs_graph in tqdm(dimacs_graphs):
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    res = fit_model(edges, vertices_num)
    real_max_cliques.append([dimacs_graph, res.objective_value])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
real_max_cliques

In [78]:
real_max_cliques

[['brock200_4.clq', 17.0],
 ['brock200_3.clq', 15.0],
 ['brock200_2.clq', 12.0],
 ['brock200_1.clq', 21.0],
 ['c-fat200-1.clq', 12.0],
 ['c-fat200-2.clq', 24.0],
 ['c-fat200-5.clq', 58.0],
 ['c-fat500-1.clq', 14.0],
 ['c-fat500-10.clq', 126.0],
 ['c-fat500-2.clq', 26.0],
 ['c-fat500-5.clq', 64.0],
 ['san200_0.7_1.clq', 29.999999999999996],
 ['san200_0.7_2.clq', 18.0],
 ['san200_0.9_1.clq', 70.0],
 ['san200_0.9_2.clq', 60.0],
 ['san200_0.9_3.clq', 44.0],
 ['sanr200_0.7.clq', 18.0],
 ['MANN_a9.clq', 16.0],
 ['hamming6-2.clq', 32.0],
 ['hamming6-4.clq', 4.0],
 ['gen200_p0.9_44.clq', 44.0],
 ['gen200_p0.9_55.clq', 55.0],
 ['C125.9.clq', 34.0],
 ['keller4.clq', 11.0],
 ['p_hat300-1.clq', 8.0],
 ['p_hat300-2.clq', 25.0]]

Все ок, что целочисленный алгоритм, что branch and bound нашли то же самое решение, так что все хорошо (если я ничего не пропустил)